# Calibrate the HBV model with your region!


After that you can manually calibrate the the model to fit your region and maybe even calibrate it for the research you want to do already!



## Importing modules

Starting with all the libraries and utility functions. We have some general Python packages that we use.
And then the eWaterCycle packages, this is where we import our 'interface', models & our way of loading generalised forcings for all supported models.
Also I import some utility functions just for this notebook that I made to make it easier.


In [1]:
# General python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import importlib
import subprocess
import sys
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import os
from IPython.display import display

# Niceties
from rich import print

# General eWaterCycle
import ewatercycle
import ewatercycle.models
import ewatercycle.forcing

# Check and install geopandas and folium
def install_if_missing(package_name):
    try:
        importlib.import_module(package_name)
        print(f"'{package_name}' is already installed.")
    except ImportError:
        print(f"'{package_name}' not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"PLEASE RESTART KERNEL")

install_if_missing("geopandas")
install_if_missing("folium")

import geopandas as gpd
import folium
import json

# Utility functions
from util_functions import *

'geopandas' is already installed.

'folium' is already installed.

## Calibration the HBV model

There are several ways to calibrate the model:
1. This simple interactive way, as you will see it quite hard to calibrate it properly.
2. Use a calibration algorithm, but this will take longer (and is less fun :) )

We are going to use a 'local' HBV model for this calibration, meaning that this is run in the cell itself, so it will not start up a container and work in the normal ways eWaterCyle works.

In [2]:
with open("settings.json", "r") as json_file:
    settings = json.load(json_file)

In [3]:
# Load the caravan forcing object
caravan_data_object = ewatercycle.forcing.sources['CaravanForcing'].load(directory=settings['path_caravan'])
print(caravan_data_object)

CaravanForcing(
    start_time='1997-08-01T00:00:00Z',
    end_time='2010-07-31T00:00:00Z',
    directory=PosixPath('/home/mmelotto/ewatercycleClimateImpact/HBV/data/camelsgb_28093/caravan'),
    shape=PosixPath('/home/mmelotto/ewatercycleClimateImpact/HBV/data/camelsgb_28093/caravan/camelsgb_28093.shp'),
    filenames={
        'evspsblpot': 'camelsgb_28093_1997-08-01_2010-07-31_evspsblpot.nc',
        'pr': 'camelsgb_28093_1997-08-01_2010-07-31_pr.nc',
        'Q': 'camelsgb_28093_1997-08-01_2010-07-31_Q.nc',
        'tasmax': 'camelsgb_28093_1997-08-01_2010-07-31_tasmax.nc',
        'tas': 'camelsgb_28093_1997-08-01_2010-07-31_tas.nc',
        'tasmin': 'camelsgb_28093_1997-08-01_2010-07-31_tasmin.nc'
    }
)

In [4]:
# Load the ERA5 forcing object
load_location = Path(settings['path_ERA5']) / "work" / "diagnostic" / "script"
ERA5_forcing_object = ewatercycle.forcing.sources["LumpedMakkinkForcing"].load(directory=load_location)
print(ERA5_forcing_object)

LumpedMakkinkForcing(
    start_time='1997-08-01T00:00:00Z',
    end_time='2010-07-31T00:00:00Z',
    directory=PosixPath('/home/mmelotto/ewatercycleClimateImpact/HBV/data/camelsgb_28093/ERA5/work/diagnostic/scrip
t'),
    shape=PosixPath('/home/mmelotto/ewatercycleClimateImpact/HBV/data/camelsgb_28093/ERA5/work/diagnostic/script/ca
melsgb_28093.shp'),
    filenames={
        'pr': 'OBS6_ERA5_reanaly_1_day_pr_1997-2010.nc',
        'tas': 'OBS6_ERA5_reanaly_1_day_tas_1997-2010.nc',
        'rsds': 'OBS6_ERA5_reanaly_1_day_rsds_1997-2010.nc',
        'evspsblpot': 'Derived_Makkink_evspsblpot.nc'
    }
)

In [5]:
# Parameters for the interactive calibrating
params = {
        'I_max': {'min': 0, 'max': 10},
        'Ce': {'min': 0.1, 'max': 1},
        'Su_max': {'min': 40, 'max': 800},
        'beta': {'min': 0.5, 'max': 5},
        'P_max': {'min': 0.001, 'max': 0.3},
        'T_lag': {'min': 1, 'max': 10},
        'Kf': {'min': 0.01, 'max': 0.2},
        'Ks': {'min': 0.0001, 'max': 0.01},
        'Fm': {'min': 0.0001, 'max': 10},
    }

# Rewrite the forcing for the interactive calibration plot, this is used for the observed streamflow
calibrate_forcing = Caravan_ERA5_to_local(caravan_data_object, ERA5_forcing_object)

local_model = ewatercycle.models.HBVLocal(forcing=caravan_data_object)

# The interactive calibration plot
interactive_plot_local(local_model, calibrate_forcing, params)

Output()

In [6]:
# Add your parameters here, dont add the FM below, keep it at 0.001
parameters_found = [
    10.0,   # Imax - Interception capacity [mm]
    0.87,   # Ce - Soil runoff coefficient [-]
    592.0,  # Sumax - Max soil moisture storage [mm]
    1.4,    # Beta - Shape parameter for runoff generation [-]
    0.3,    # Pmax - Percolation threshold [mm/day]
    1.0,    # Tlag - Routing lag time [days]
    0.09,   # Kf - Fast runoff recession coefficient [1/day]
    0.01,   # Ks - Slow runoff recession coefficient [1/day]
    1.0   # FM - Snowmelt factor 
]

In [7]:
# Still we have to kill the container
local_model.finalize()